In [ ]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import glob
import json

In [ ]:
from scipy.ndimage import uniform_filter1d

In [ ]:
hf_path = "/home/joosep/huggingface/particleflow/"

In [ ]:
df1 = pandas.read_csv(hf_path + "/clic/clusters/v2.3.0/pyg-clic_20250204_081614_352844/pyg-clic_20250204_081614_352844_runs_train.csv")
df2 = pandas.read_csv(hf_path + "/clic/clusters/v2.3.0/pyg-clic_20250211_145811_219129/pyg-clic_20250211_145811_219129_runs_train.csv")
df3 = pandas.read_csv(hf_path + "/clic/clusters/v2.3.0/pyg-clic_20250130_214007_333962/pyg-clic_20250130_214007_333962_runs_train.csv")

df1["RelTime"] = (df1["Wall time"] - df1["Wall time"][0])/3600
df2["RelTime"] = (df2["Wall time"] - df2["Wall time"][0])/3600
df3["RelTime"] = (df3["Wall time"] - df3["Wall time"][0])/3600

time_window = 20
normval = uniform_filter1d(df3["Value"], time_window)[-1]
df1["Value"] = df1["Value"]/normval
df2["Value"] = df2["Value"]/normval
df3["Value"] = df3["Value"]/normval

In [ ]:
plt.figure(figsize=(6,3))
p = plt.plot(df1["RelTime"], df1["Value"], alpha=0.2)
plt.plot(df1["RelTime"], uniform_filter1d(df1["Value"], time_window), color=p[0].get_color(), label="GNNLSH")

p = plt.plot(df2["RelTime"], df2["Value"], alpha=0.2)
plt.plot(df2["RelTime"], uniform_filter1d(df2["Value"], time_window), color=p[0].get_color(), label="full transformer")

p = plt.plot(df3["RelTime"], df3["Value"], alpha=0.2)
plt.plot(df3["RelTime"], uniform_filter1d(df3["Value"], time_window), color=p[0].get_color(), label="full transformer+FlashAttention")

plt.xlim(-1,75)
plt.ylim(0.5, 4)

plt.legend(loc="best", frameon=False)
plt.ylabel("Relative training loss")
plt.xlabel("Training time [hours]")
plt.savefig("step_train_loss.pdf", bbox_inches="tight")

In [ ]:
def load_history(path, min_epoch=None, max_epoch=None):
    ret = {}
    for fi in glob.glob(path):
        data = json.load(open(fi))
        data2 = {}
        for k1 in ["train", "valid"]:
            for k2 in data[k1].keys():
                data2[f"{k1}_{k2}"] = data[k1][k2]
        epoch = int(fi.split("_")[-1].split(".")[0])
        ret[epoch] = data2

    if not max_epoch:
        max_epoch = max(ret.keys())
    if not min_epoch:
        min_epoch = min(ret.keys())

    ret2 = []
    for i in range(min_epoch, max_epoch + 1):
        ret2.append(ret[i])
    
    return pandas.DataFrame(ret2)

In [ ]:
history = load_history(hf_path + "clic/clusters/v2.2.0/pyg-clic_20250106_193536_269746/history/epoch_*.json", max_epoch=5)
# history = load_history(hf_path + "clic/clusters/v2.3.0/pyg-clic_20250130_214007_333962/history/epoch_*.json", max_epoch=10)

In [ ]:
val_reg_loss = np.array(sum(
    [history["valid_Regression_{}".format(l)].values for l in ["energy", "pt", "eta", "sin_phi", "cos_phi"]]
))

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(history.index, history["valid_Total"], marker="s", label="total")
plt.plot(history.index, history["valid_Classification_binary"], marker="v", label="primary classification")
plt.plot(history.index, 100*history["valid_Classification"], marker="^", label="PID classification x100")
plt.plot(history.index, 10*val_reg_loss, marker="o", label="regression x10")
plt.legend(loc="best", frameon=False)
plt.ylim(1,3)
plt.ylabel("Validation loss")
plt.xticks(range(0,5), range(1,6))
plt.xlabel("Training epoch")
#plt.yscale("log")
plt.savefig("epoch_valid_loss.pdf", bbox_inches="tight")